# Link Recommendation for Reducing Polarization in Social Networks

## Basic Classes for Calculation

In [14]:
import networkx as nx
import numpy


class PolarizationGraph:
    polarization = 0

    def __init__(self, graph: nx.Graph, internal_opinions: list):
        self.nx_graph = graph
        # the internal_opinions list contains a dict for each node in the form {"value" : x, "polarization": y}
        self.internal_opinions = internal_opinions
        self.external_opinions = [
            {"value": x["value"], "polarization": 0} for x in internal_opinions]

    def setExternal(self, new_external: list):
        self.external_opinions = new_external


class PolarizationHandler:
    def calcExternalOpinions(graph: PolarizationGraph) -> list:
        expressed = []

        for iter in range(1000):
            for n in graph.nx_graph.nodes():
                node_name = n if isinstance(n, int) else n[0]
                my_internal = next(
                    x["polarization"] for x in graph.internal_opinions if x["value"] == node_name)
                my_neighbours = list(nx.neighbors(graph.nx_graph, n))
                external_neighbours = [
                    x["polarization"] for x in graph.external_opinions if x["value"] in my_neighbours]
                my_expressed = (my_internal + numpy.sum(external_neighbours)) / \
                    (1 + numpy.sum([1 for neigh in my_neighbours]))

                if len(expressed) != len(graph.external_opinions):
                    expressed.append(
                        {"value": node_name, "polarization": my_expressed})
                else:
                    my_set = next(
                        s for s in expressed if s["value"] == node_name)
                    my_set["polarization"] = my_expressed

            graph.setExternal(expressed)
            # print(PolarizationHandler.calcPolarization(graph))

        return expressed

    def calcPolarization(graph: PolarizationGraph) -> float:
        polArr = [x["polarization"] for x in graph.external_opinions]
        nbrNodes = nx.number_of_nodes(graph.nx_graph)
        return numpy.linalg.norm(polArr) / nbrNodes

class HeuristicsHandler:
    def getExtremeExpressed(graph: PolarizationGraph, nbr_of_nodes_to_neutralize=None) -> list:
        external_opinions = graph.external_opinions
        
        sorted_list = sorted(external_opinions, key=lambda x: x["polarization"], reverse=True)
        
        if nbr_of_nodes_to_neutralize is None:
            return sorted_list
        else:
            return sorted_list[0:nbr_of_nodes_to_neutralize]
        """
        iter = 0
        while (iter < nbr_of_nodes_to_neutralize):
            most_extreme_opinion = 0
            node_to_neutralize = 0
            for x in external_opinions:
                current_abs_opinion = abs(x["polarization"])
                if current_abs_opinion > most_extreme_opinion:
                    most_extreme_opinion = current_abs_opinion
                    node_to_neutralize = x["value"]
            external_opinions[node_to_neutralize - 1] = {"value": node_to_neutralize, "polarization": 0}
            graph.setExternal(PolarizationHandler.calcExternalOpinions(graph))
            external_opinions = graph.external_opinions
            iter+=1
        return graph
        """
        
    def getExtremeNeighbours(graph: PolarizationGraph, nbr_extreme_neighborhoods = None) -> list:
        neighbourhood_extremities = []
        for n in graph.nx_graph.nodes():
            n_neighbours = list(nx.neighbors(graph.nx_graph, n))
            external_neighbours = [x["polarization"] for x in graph.external_opinions if x["value"] in n_neighbours]
            n_neighbourhood_extremity = sum(map(abs, external_neighbours))
            n_dict = {"value": n, "neighbourhood_extremity": n_neighbourhood_extremity}
            neighbourhood_extremities.append(n_dict)
        
        neighbourhood_extremities = sorted(neighbourhood_extremities, key=lambda x: x["neighbourhood_extremity"], reverse=True)
        
        if nbr_extreme_neighborhoods is None:
            return neighbourhood_extremities
        else:
            return neighbourhood_extremities[0:nbr_extreme_neighborhoods]

# Data Setup

In [16]:
import networkx as nx

KarateGraph = nx.read_gml("karate.gml", None)
BooksGraph = nx.read_gml("polbooks.gml", None)

ground_truth_as_sets = [{20, 1, 2, 22, 3, 5, 4, 7, 6, 8, 11, 13, 12, 14, 18, 10, 17},
                        {24, 26, 28, 33, 30, 34, 25, 32, 27, 21, 23, 29, 9, 31, 15, 16, 19}]

internal_karate = []
internal_books = []

for i in range(1, 35):
    polarization = -1 if i in ground_truth_as_sets[0] else 1
    internal_karate.append({"value": i, "polarization": polarization})

for n in BooksGraph.nodes.data():
    polarization = 0 if n[1]["value"] == "n" else - \
        1 if n[1]["value"] == "c" else 1
    internal_books.append({"value": n[0], "polarization": polarization})

KaratePolGraph = PolarizationGraph(KarateGraph, internal_karate)
BooksPolGraph = PolarizationGraph(BooksGraph, internal_books)

KaratePolGraph.setExternal(PolarizationHandler.calcExternalOpinions(KaratePolGraph))
print(PolarizationHandler.calcPolarization(KaratePolGraph))


for gr in [
    KaratePolGraph, 
    BooksPolGraph
]:
    exprOpinions = PolarizationHandler.calcExternalOpinions(gr)
    gr.setExternal(exprOpinions)
    gr.polarization = PolarizationHandler.calcPolarization(gr)
    print(gr.polarization)

nodes_to_influence = HeuristicsHandler.getExtremeNeighbours(KaratePolGraph)
print(nodes_to_influence)

0.10221847999166145
0.10221847999166145
0.06431294911675486
[{'value': 1, 'neighbourhood_extremity': 8.897097452788781}, {'value': 34, 'neighbourhood_extremity': 8.89649868858028}, {'value': 33, 'neighbourhood_extremity': 6.3304831347863475}, {'value': 3, 'neighbourhood_extremity': 4.352280005500802}, {'value': 2, 'neighbourhood_extremity': 4.087697906008064}, {'value': 32, 'neighbourhood_extremity': 3.2781979877740506}, {'value': 6, 'neighbourhood_extremity': 2.8750769768279167}, {'value': 7, 'neighbourhood_extremity': 2.8750769768279167}, {'value': 24, 'neighbourhood_extremity': 2.859869905624605}, {'value': 4, 'neighbourhood_extremity': 2.650221862322876}, {'value': 30, 'neighbourhood_extremity': 2.3256729502871853}, {'value': 14, 'neighbourhood_extremity': 2.056237687933689}, {'value': 5, 'neighbourhood_extremity': 2.0250667132508613}, {'value': 11, 'neighbourhood_extremity': 2.0250667132508613}, {'value': 9, 'neighbourhood_extremity': 2.0028638636068794}, {'value': 28, 'neighbourh